In [1]:
!pip install ogb torch-geometric

import torch
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.loader import NeighborLoader
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, GCNConv
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 836.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [1]:
# بارگذاری دیتاست
dataset = PygNodePropPredDataset(name='ogbn-products')
data = dataset[0]

print(f"تعداد نودها: {data.num_nodes}")
print(f"تعداد یالها: {data.num_edges}")
print(f"تعداد ویژگی‌های هر نود: {data.num_node_features}")
print(f"تعداد کلاس‌ها: {dataset.num_classes}")

# تقسیم داده به train/val/test
split_idx = dataset.get_idx_split()
train_idx = split_idx['train']
val_idx = split_idx['valid']
test_idx = split_idx['test']

In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

# Train for Node Pr

In [ ]:
def train(model, data, train_idx, optimizer, criterion):
    model.train()
    optimizer.zero_grad()

    out = model(data.x, data.edge_index)[train_idx]
    loss = criterion(out, data.y[train_idx].view(-1))

    loss.backward()
    optimizer.step()

    return loss.item()

def evaluate(model, data, idx):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)[idx]
        pred = out.argmax(dim=1)
        true = data.y[idx].view(-1)

        acc = accuracy_score(true.cpu(), pred.cpu())
        f1 = f1_score(true.cpu(), pred.cpu(), average='weighted')

    return acc, f1

In [ ]:
# پارامترها
in_channels = data.num_node_features
hidden_channels = 256
out_channels = dataset.num_classes
epochs = 100
lr = 0.01

# مدل، optimizer و loss function
sage_model = GraphSAGE(in_channels, hidden_channels, out_channels)
optimizer = torch.optim.Adam(sage_model.parameters(), lr=lr)
criterion = torch.nn.NLLLoss()

# آموزش
sage_train_loss = []
sage_val_acc = []
sage_val_f1 = []

for epoch in range(epochs):
    loss = train(sage_model, data, train_idx, optimizer, criterion)
    acc, f1 = evaluate(sage_model, data, val_idx)

    sage_train_loss.append(loss)
    sage_val_acc.append(acc)
    sage_val_f1.append(f1)

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}, Val Acc: {acc:.4f}, Val F1: {f1:.4f}')

In [ ]:
gcn_model = GCN(in_channels, hidden_channels, out_channels)
optimizer = torch.optim.Adam(gcn_model.parameters(), lr=lr)

gcn_train_loss = []
gcn_val_acc = []
gcn_val_f1 = []

for epoch in range(epochs):
    loss = train(gcn_model, data, train_idx, optimizer, criterion)
    acc, f1 = evaluate(gcn_model, data, val_idx)

    gcn_train_loss.append(loss)
    gcn_val_acc.append(acc)
    gcn_val_f1.append(f1)

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}, Val Acc: {acc:.4f}, Val F1: {f1:.4f}')

In [ ]:
# ارزیابی روی داده تست
sage_test_acc, sage_test_f1 = evaluate(sage_model, data, test_idx)
gcn_test_acc, gcn_test_f1 = evaluate(gcn_model, data, test_idx)

print(f'GraphSAGE - Test Accuracy: {sage_test_acc:.4f}, Test F1: {sage_test_f1:.4f}')
print(f'GCN - Test Accuracy: {gcn_test_acc:.4f}, Test F1: {gcn_test_f1:.4f}')

# رسم نمودارها
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(sage_train_loss, label='GraphSAGE')
plt.plot(gcn_train_loss, label='GCN')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(sage_val_acc, label='GraphSAGE Val Acc')
plt.plot(gcn_val_acc, label='GCN Val Acc')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Edge Pr

In [ ]:
from torch_geometric.nn import LinkPrediction
from torch_geometric.utils import negative_sampling

class EdgePredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.lin1 = torch.nn.Linear(2 * in_channels, 128)
        self.lin2 = torch.nn.Linear(128, 1)

    def forward(self, z, edge_index):
        src, dst = edge_index
        x = torch.cat([z[src], z[dst]], dim=1)
        x = F.relu(self.lin1(x))
        return torch.sigmoid(self.lin2(x)).view(-1)

# استفاده از embeddings یادگرفته شده توسط GraphSAGE
edge_model = EdgePredictor(hidden_channels)
optimizer = torch.optim.Adam(edge_model.parameters(), lr=0.01)

# آموزش edge prediction
for epoch in range(50):
    edge_model.train()
    optimizer.zero_grad()

    # تولید embeddings با GraphSAGE
    z = sage_model.conv1(data.x, data.edge_index)
    z = sage_model.conv2(z, data.edge_index)

    # نمونه‌گیری از یالهای مثبت و منفی
    pos_edge_index = data.edge_index
    neg_edge_index = negative_sampling(data.edge_index, num_nodes=data.num_nodes)

    # محاسبه loss
    pos_pred = edge_model(z, pos_edge_index)
    neg_pred = edge_model(z, neg_edge_index)

    pos_loss = -torch.log(pos_pred + 1e-15).mean()
    neg_loss = -torch.log(1 - neg_pred + 1e-15).mean()
    loss = pos_loss + neg_loss

    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

-----------------------------
# Temp

In [1]:
# نصب numpy سازگار
!pip install numpy==1.24.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 100.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.


In [1]:
# نصب نسخه سازگار PyTorch
!pip install torch==2.0.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# نصب کتابخانه‌های وابسته دقیقاً مطابق PyG توصیه‌شده برای torch==2.0.1
!pip install pyg-lib==0.2.0 torch-scatter==2.1.1 torch-sparse==0.6.17 torch-cluster==1.6.1 torch-spline-conv==1.2.2 -f https://data.pyg.org/whl/torch-2.0.1+cu118.html

# نصب نسخه صحیح PyG
!pip install torch-geometric==2.3.1

# نصب OGB
!pip install ogb


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 114.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 105.0 MB/s eta 0:00:00
INFO: pip is still looki

In [ ]:
# # نصب نسخه صحیح torch و torchvision
# !pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118

# # نصب وابستگی‌های PyTorch Geometric
# !pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html

# # نصب نسخه مناسب PyG
# !pip install torch-geometric==2.3.1

# # نصب ogb
# !pip install ogb


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89991 sha256=119bc8333b7814b2efa7f68fecd15c3053cee16486b10961e5405e19b0b5f7a0
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cpu
    Uninstalling torch-2.6.0+cpu:
      Successfully unins

In [5]:
import torch
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric.utils import to_undirected

# بارگذاری دیتاست ogbn-products
dataset = PygNodePropPredDataset(name='ogbn-products')
data = dataset[0]  # فقط یک شیء Data برمی‌گرداند

# یال‌ها را بدون جهت می‌کنیم
data.edge_index = to_undirected(data.edge_index)

# تبدیل برچسب‌ها به [num_nodes]
data.y = data.y.squeeze()

# ماسک‌های آموزش، اعتبارسنجی، آزمون
split_idx = dataset.get_idx_split()
train_idx = split_idx['train']
val_idx = split_idx['valid']
test_idx = split_idx['test']

# بررسی اولیه
print(data)
print(f"# Train samples: {train_idx.shape[0]}")


Data(num_nodes=2449029, edge_index=[2, 123718152], x=[2449029, 100], y=[2449029])
# Train samples: 196615


### سریع ساز:

In [6]:
from torch_geometric.loader import NeighborLoader
from torch import tensor

# data = dataset[0]
# data.edge_index = to_undirected(data.edge_index)
# data.y = data.y.squeeze()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# # فقط همین بخش روی GPU یا CPU منتقل میشه
# data = data.to(device)

train_idx = split_idx['train'].clone().detach()
val_idx = split_idx['valid'].clone().detach()
test_idx = split_idx['test'].clone().detach()

train_loader = NeighborLoader(
    data,
    input_nodes=train_idx,
    num_neighbors=[15, 10],
    batch_size=512,
    shuffle=True
)

val_loader = NeighborLoader(
    data,
    input_nodes=val_idx,
    num_neighbors=[15, 10],
    batch_size=512
)

test_loader = NeighborLoader(
    data,
    input_nodes=test_idx,
    num_neighbors=[15, 10],
    batch_size=512
)





## GCN

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# تعریف مدل GCN دو لایه
class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        # لایه اول: از ویژگی‌ها به فضای پنهان
        self.conv1 = GCNConv(num_features, hidden_channels)
        # لایه دوم: از فضای پنهان به کلاس‌ها
        self.conv2 = GCNConv(hidden_channels, num_classes)

    # def forward(self, data):
    #     x, edge_index = data.x, data.edge_index
    #     x = self.conv1(x, edge_index)
    #     x = F.relu(x)
    #     x = F.dropout(x, p=0.5, training=self.training)
    #     x = self.conv2(x, edge_index)
    #     return x

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x



In [ ]:
# مقادیر از دیتاست
num_features = data.num_node_features       # =100
num_classes = int(data.y.max().item()) + 1  # =47
hidden_channels = 64                       # قابل تنظیم

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ساخت مدل
model = GCN(num_features, hidden_channels, num_classes).to(device)
data = data.to(device)
train_idx = train_idx.to(device)
val_idx = val_idx.to(device)
test_idx = test_idx.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# def train():
#     model.train()
#     optimizer.zero_grad()
#     out = model(data)
#     loss = F.cross_entropy(out[train_idx], data.y[train_idx])
#     loss.backward()
#     optimizer.step()
#     return loss.item()
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = F.cross_entropy(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


# @torch.no_grad()
# def test():
#     model.eval()
#     out = model(data)
#     pred = out.argmax(dim=1)

#     accs = []
#     for idx in [train_idx, val_idx, test_idx]:
#         correct = (pred[idx] == data.y[idx]).sum().item()
#         acc = correct / idx.shape[0]
#         accs.append(acc)
#     return accs
@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    total = 0
    for batch in loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index)
        pred = out.argmax(dim=1)
        correct += (pred == batch.y).sum().item()
        total += batch.y.size(0)
    return correct / total


for epoch in range(1, 3):
    loss = train()
    train_acc = test(train_loader)
    val_acc = test(val_loader)
    test_acc = test(test_loader)
    print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, "
          f"Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}")
    torch.save(model.state_dict(), f"gcn_epoch_{epoch:03d}.pt")



### Crach & no run

In [ ]:
# مقادیر از دیتاست
num_features = data.num_node_features       # =100
num_classes = int(data.y.max().item()) + 1  # =47
hidden_channels = 128                       # قابل تنظیم

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ساخت مدل
model = GCN(num_features, hidden_channels, num_classes).to(device)
data = data.to(device)
train_idx = train_idx.to(device)
val_idx = val_idx.to(device)
test_idx = test_idx.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# def train():
#     model.train()
#     optimizer.zero_grad()
#     out = model(data)
#     loss = F.cross_entropy(out[train_idx], data.y[train_idx])
#     loss.backward()
#     optimizer.step()
#     return loss.item()
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = F.cross_entropy(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


# @torch.no_grad()
# def test():
#     model.eval()
#     out = model(data)
#     pred = out.argmax(dim=1)

#     accs = []
#     for idx in [train_idx, val_idx, test_idx]:
#         correct = (pred[idx] == data.y[idx]).sum().item()
#         acc = correct / idx.shape[0]
#         accs.append(acc)
#     return accs
@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    total = 0
    for batch in loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index)
        pred = out.argmax(dim=1)
        correct += (pred == batch.y).sum().item()
        total += batch.y.size(0)
    return correct / total


for epoch in range(1, 31):  # فقط 30 دوره برای سرعت اولیه
    loss = train()
    train_acc = test(train_loader)
    val_acc = test(val_loader)
    test_acc = test(test_loader)
    print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, "
          f"Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}")



Epoch 001, Loss: 1.4965, Train: 0.6650, Val: 0.6638, Test: 0.5794


## GraphSAGE

In [ ]:
from torch_geometric.nn import SAGEConv

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    # def forward(self, data):
    #     x, edge_index = data.x, data.edge_index
    #     x = self.conv1(x, edge_index)
    #     x = F.relu(x)
    #     x = F.dropout(x, p=0.5, training=self.training)
    #     x = self.conv2(x, edge_index)
    #     return x

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x


In [ ]:
sage_model = GraphSAGE(num_features, hidden_channels, num_classes).to(device)
optimizer = torch.optim.Adam(sage_model.parameters(), lr=0.01, weight_decay=5e-4)

def train_sage():
    sage_model.train()
    optimizer.zero_grad()
    out = sage_model(data)
    loss = F.cross_entropy(out[train_idx], data.y[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test_sage():
    sage_model.eval()
    out = sage_model(data)
    pred = out.argmax(dim=1)
    accs = []
    for idx in [train_idx, val_idx, test_idx]:
        correct = (pred[idx] == data.y[idx]).sum().item()
        acc = correct / idx.shape[0]
        accs.append(acc)
    return accs


sage_train_acc_list = []
sage_val_acc_list = []
sage_test_acc_list = []

for epoch in range(1, 3):
    loss = train_sage()
    train_acc, val_acc, test_acc = test_sage()

    sage_train_acc_list.append(train_acc)
    sage_val_acc_list.append(val_acc)
    sage_test_acc_list.append(test_acc)

    if epoch % 10 == 0:
        print(f"[GraphSAGE] Epoch {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}")


# Eval

### Acc

In [ ]:
import matplotlib.pyplot as plt

# فرض: دقت‌های هر مدل در طول آموزش قبلاً جمع‌آوری شده‌اند
# این مقادیر برای نمایش نمونه‌ای هستند و در اجرای واقعی باید از متغیرهای خروجی واقعی مثل `train_acc_list` و `sage_train_acc_list` استفاده شوند.
epochs = list(range(1, 201))

# داده‌های فرضی برای GCN
gcn_train_acc = [0.50 + 0.002*e for e in epochs]
gcn_val_acc = [0.48 + 0.0018*e for e in epochs]
gcn_test_acc = [0.47 + 0.0015*e for e in epochs]

# داده‌های فرضی برای GraphSAGE
sage_train_acc = [0.52 + 0.0021*e for e in epochs]
sage_val_acc = [0.50 + 0.0019*e for e in epochs]
sage_test_acc = [0.49 + 0.0017*e for e in epochs]

# رسم نمودار مقایسه‌ای دقت مدل‌ها
plt.figure(figsize=(12, 6))
plt.plot(epochs, gcn_train_acc, label='GCN Train Acc')
plt.plot(epochs, gcn_val_acc, label='GCN Val Acc')
plt.plot(epochs, gcn_test_acc, label='GCN Test Acc')

plt.plot(epochs, sage_train_acc, label='SAGE Train Acc', linestyle='--')
plt.plot(epochs, sage_val_acc, label='SAGE Val Acc', linestyle='--')
plt.plot(epochs, sage_test_acc, label='SAGE Test Acc', linestyle='--')

plt.title('Accuracy Comparison Between GCN and GraphSAGE')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### F1-Score

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

# تولید داده نمونه‌ای بدون نیاز به torch
np.random.seed(42)
y_true = np.random.randint(0, 47, 10000)
y_pred_gcn = y_true.copy()
y_pred_sage = y_true.copy()

# افزودن خطا برای ایجاد تفاوت عملکرد
y_pred_gcn[::10] = np.random.randint(0, 47, 1000)
y_pred_sage[::15] = np.random.randint(0, 47, 667)

# محاسبه F1-score
f1_macro_gcn = f1_score(y_true, y_pred_gcn, average='macro')
f1_micro_gcn = f1_score(y_true, y_pred_gcn, average='micro')

f1_macro_sage = f1_score(y_true, y_pred_sage, average='macro')
f1_micro_sage = f1_score(y_true, y_pred_sage, average='micro')

# نمایش نمودار
labels = ['GCN Macro', 'GCN Micro', 'SAGE Macro', 'SAGE Micro']
scores = [f1_macro_gcn, f1_micro_gcn, f1_macro_sage, f1_micro_sage]

plt.figure(figsize=(8,5))
plt.bar(labels, scores, color=['orange', 'red', 'blue', 'cyan'])
plt.title("F1-Score Comparison Between GCN and GraphSAGE (Simulated)")
plt.ylabel("F1 Score")
plt.ylim(0, 1)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
# 5. ارزیابی نهایی
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)

        train_acc = accuracy_score(data.y[data.train_mask].cpu(),
                                 pred[data.train_mask].cpu())
        val_acc = accuracy_score(data.y[data.val_mask].cpu(),
                               pred[data.val_mask].cpu())
        test_acc = accuracy_score(data.y[data.test_mask].cpu(),
                                pred[data.test_mask].cpu())

        test_f1 = f1_score(data.y[data.test_mask].cpu(),
                         pred[data.test_mask].cpu(), average='weighted')

        return train_acc, val_acc, test_acc, test_f1

print("\nارزیابی GraphSAGE:")
sage_train_acc, sage_val_acc, sage_test_acc, sage_test_f1 = evaluate_model(sage_model, data)
print(f"Train Acc: {sage_train_acc:.4f}, Val Acc: {sage_val_acc:.4f}, "
      f"Test Acc: {sage_test_acc:.4f}, Test F1: {sage_test_f1:.4f}")

print("\nارزیابی GCN:")
gcn_train_acc, gcn_val_acc, gcn_test_acc, gcn_test_f1 = evaluate_model(gcn_model, data)
print(f"Train Acc: {gcn_train_acc:.4f}, Val Acc: {gcn_val_acc:.4f}, "
      f"Test Acc: {gcn_test_acc:.4f}, Test F1: {gcn_test_f1:.4f}")

# 6. مقایسه مدل‌ها
plt.figure(figsize=(15, 10))

# نمودار خطا
plt.subplot(2, 2, 1)
plt.plot(sage_loss, label='GraphSAGE')
plt.plot(gcn_loss, label='GCN')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# نمودار دقت اعتبارسنجی
plt.subplot(2, 2, 2)
plt.plot(sage_val_acc, label='GraphSAGE')
plt.plot(gcn_val_acc, label='GCN')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# مقایسه F1
plt.subplot(2, 2, 3)
models = ['GraphSAGE', 'GCN']
test_f1 = [sage_test_f1, gcn_test_f1]
plt.bar(models, test_f1, color=['blue', 'orange'])
plt.title('Test F1-Score Comparison')
plt.ylabel('F1-Score')

# مقایسه دقت تست
plt.subplot(2, 2, 4)
test_acc = [sage_test_acc, gcn_test_acc]
plt.bar(models, test_acc, color=['blue', 'orange'])
plt.title('Test Accuracy Comparison')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.savefig('results_comparison.png')
plt.show()


In [ ]:
# 7. پیاده‌سازی اضافی: Edge Prediction
class EdgePredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(EdgePredictor, self).__init__()
        self.lin1 = torch.nn.Linear(2 * in_channels, 128)
        self.lin2 = torch.nn.Linear(128, 1)

    def forward(self, z, edge_index):
        src, dst = edge_index
        x = torch.cat([z[src], z[dst]], dim=1)
        x = F.relu(self.lin1(x))
        return torch.sigmoid(self.lin2(x)).squeeze()

def get_embeddings(model, data):
    model.eval()
    with torch.no_grad():
        # استخراج embeddings از لایه اول
        embeddings = model.conv1(data.x, data.edge_index)
        embeddings = F.relu(embeddings)
        return embeddings

print("\nآموزش مدل Edge Prediction با GraphSAGE:")
sage_embeddings = get_embeddings(sage_model, data)
edge_model = EdgePredictor(256).to(device)
optimizer = torch.optim.Adam(edge_model.parameters(), lr=0.01)

# نمونه‌گیری از یالهای منفی
def negative_sampling(edge_index, num_nodes, num_neg_samples=None):
    if num_neg_samples is None:
        num_neg_samples = edge_index.size(1)

    neg_edge_index = torch.randint(0, num_nodes, (2, num_neg_samples), device=device)
    return neg_edge_index

for epoch in range(50):
    edge_model.train()
    optimizer.zero_grad()

    # پیش‌بینی برای یالهای مثبت
    pos_pred = edge_model(sage_embeddings, data.edge_index)
    pos_loss = F.binary_cross_entropy(pos_pred, torch.ones_like(pos_pred))

    # نمونه‌گیری و پیش‌بینی برای یالهای منفی
    neg_edge_index = negative_sampling(data.edge_index, data.num_nodes, num_neg_samples=data.edge_index.size(1))
    neg_pred = edge_model(sage_embeddings, neg_edge_index)
    neg_loss = F.binary_cross_entropy(neg_pred, torch.zeros_like(neg_pred))

    loss = pos_loss + neg_loss
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/50, Loss: {loss.item():.4f}')

# ارزیابی Edge Prediction
edge_model.eval()
with torch.no_grad():
    pos_pred = edge_model(sage_embeddings, data.edge_index)
    neg_edge_index = negative_sampling(data.edge_index, data.num_nodes, num_neg_samples=100000)
    neg_pred = edge_model(sage_embeddings, neg_edge_index)

    # محاسبه دقت
    pos_acc = (pos_pred > 0.5).float().mean()
    neg_acc = (neg_pred < 0.5).float().mean()
    overall_acc = (pos_acc * pos_pred.size(0) + neg_acc * neg_pred.size(0)) / (pos_pred.size(0) + neg_pred.size(0))

    print(f"\nنتایج Edge Prediction:")
    print(f"Positive Accuracy: {pos_acc.item():.4f}")
    print(f"Negative Accuracy: {neg_acc.item():.4f}")
    print(f"Overall Accuracy: {overall_acc.item():.4f}")

# Extra Point

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.utils import to_undirected, train_test_split_edges, negative_sampling
from torch_geometric.nn import GCNConv
from ogb.nodeproppred import PygNodePropPredDataset
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score

# 1. بارگذاری دیتاست و آماده‌سازی گراف
# dataset = PygNodePropPredDataset(name='ogbn-products')
# data = dataset[0]
data.edge_index = to_undirected(data.edge_index)
data.y = data.y.squeeze()

# 2. تبدیل گراف برای وظیفه پیش‌بینی یال
data = train_test_split_edges(data, val_ratio=0.05, test_ratio=0.2)

# 3. مدل GCN Encoder برای تولید embedding نودها
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# 4. تابع پیش‌بینی یال (inner product decoder)
def decode(z, edge_index):
    return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)

# 5. تابع محاسبه loss با نمونه‌گیری منفی
def compute_loss(z, pos_edge_index, num_nodes, batch_size=100000):
    pos_score = decode(z, pos_edge_index)
    pos_loss = -F.logsigmoid(pos_score).mean()

    neg_score_sum = 0
    neg_batches = (pos_edge_index.size(1) // batch_size) + 1

    for _ in range(neg_batches):
        neg_edge_index = negative_sampling(
            edge_index=pos_edge_index,
            num_nodes=num_nodes,
            num_neg_samples=min(batch_size, pos_edge_index.size(1)),
        )
        neg_score = decode(z, neg_edge_index)
        neg_score_sum += -F.logsigmoid(-neg_score).mean()

    return pos_loss + neg_score_sum / neg_batches

# 6. تنظیمات مدل و داده‌ها
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNEncoder(data.num_node_features, 64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x = data.x.to(device)
train_edge_index = data.train_pos_edge_index.to(device)

# 7. حلقه آموزش
def train_edge_prediction():
    model.train()
    optimizer.zero_grad()
    z = model(x, train_edge_index)
    loss = compute_loss(z, train_edge_index, x.size(0))
    loss.backward()
    optimizer.step()
    return loss.item()

# 8. اجرای چند اپوک آموزشی سبک
for epoch in range(1, 3):
    loss = train_edge_prediction()
    print(f"[EdgePred] Epoch {epoch:02d} | Loss: {loss:.4f}")


NameError: name 'data' is not defined

In [ ]:
@torch.no_grad()
def evaluate_edge_prediction(model, x, edge_index, pos_edge_index, neg_edge_index):
    model.eval()
    z = model(x, edge_index)
    pos_scores = torch.sigmoid(decode(z, pos_edge_index)).cpu().numpy()
    neg_scores = torch.sigmoid(decode(z, neg_edge_index)).cpu().numpy()
    y_true = np.hstack([np.ones(pos_scores.shape[0]), np.zeros(neg_scores.shape[0])])
    y_scores = np.hstack([pos_scores, neg_scores])
    auc = roc_auc_score(y_true, y_scores)
    ap = average_precision_score(y_true, y_scores)
    return auc, ap

full_edge_index = data.edge_index.to(device)
val_pos = data.val_pos_edge_index.to(device)
val_neg = negative_sampling(full_edge_index, x.size(0), num_neg_samples=val_pos.size(1))

test_pos = data.test_pos_edge_index.to(device)
test_neg = negative_sampling(full_edge_index, x.size(0), num_neg_samples=test_pos.size(1))

val_auc, val_ap = evaluate_edge_prediction(model, x, full_edge_index, val_pos, val_neg)
test_auc, test_ap = evaluate_edge_prediction(model, x, full_edge_index, test_pos, test_neg)

print(f"Validation AUC: {val_auc:.4f}, AP: {val_ap:.4f}")
print(f"Test AUC: {test_auc:.4f}, AP: {test_ap:.4f}")


##-----------------------------------------

In [ ]:
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install ogb

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from ogb.nodeproppred import NodePropPredDataset
from torch_geometric.nn import GCNConv, SAGEConv
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

# 1. آماده‌سازی داده‌ها
dataset = NodePropPredDataset(name='ogbn-products')
split_idx = dataset.get_idx_split()

graph, labels = dataset[0]
edge_index = torch.tensor(graph['edge_index'], dtype=torch.long)
x = torch.tensor(graph['node_feat'], dtype=torch.float)
y = torch.tensor(labels, dtype=torch.long).squeeze()

# ایجاد ماسک‌های آموزشی، اعتبارسنجی و تست
data = Data(x=x, edge_index=edge_index, y=y)
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

data.train_mask[split_idx["train"]] = True
data.val_mask[split_idx["valid"]] = True
data.test_mask[split_idx["test"]] = True

# 2. تعریف مدل‌ها
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# 3. تنظیمات آموزش
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

def train_model(model, data, epochs=100):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    train_losses, val_accs, val_f1s = [], [], []

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # ارزیابی
        model.eval()
        with torch.no_grad():
            out = model(data)
            pred = out.argmax(dim=1)

            # محاسبه دقت و F1 برای اعتبارسنجی
            val_acc = accuracy_score(data.y[data.val_mask].cpu(),
                                   pred[data.val_mask].cpu())
            val_f1 = f1_score(data.y[data.val_mask].cpu(),
                             pred[data.val_mask].cpu(), average='weighted')

        train_losses.append(loss.item())
        val_accs.append(val_acc)
        val_f1s.append(val_f1)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, '
                  f'Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

    return model, train_losses, val_accs, val_f1s

# 4. آموزش مدل‌ها
print("\nآموزش مدل GraphSAGE:")
sage_model, sage_loss, sage_val_acc, sage_val_f1 = train_model(
    GraphSAGE(data.num_features, 256, dataset.num_classes),
    data
)

print("\nآموزش مدل GCN:")
gcn_model, gcn_loss, gcn_val_acc, gcn_val_f1 = train_model(
    GCN(data.num_features, 256, dataset.num_classes),
    data
)

# 5. ارزیابی نهایی
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data)
        pred = out.argmax(dim=1)

        train_acc = accuracy_score(data.y[data.train_mask].cpu(),
                                 pred[data.train_mask].cpu())
        val_acc = accuracy_score(data.y[data.val_mask].cpu(),
                               pred[data.val_mask].cpu())
        test_acc = accuracy_score(data.y[data.test_mask].cpu(),
                                pred[data.test_mask].cpu())

        test_f1 = f1_score(data.y[data.test_mask].cpu(),
                         pred[data.test_mask].cpu(), average='weighted')

        return train_acc, val_acc, test_acc, test_f1

print("\nارزیابی GraphSAGE:")
sage_train_acc, sage_val_acc, sage_test_acc, sage_test_f1 = evaluate_model(sage_model, data)
print(f"Train Acc: {sage_train_acc:.4f}, Val Acc: {sage_val_acc:.4f}, "
      f"Test Acc: {sage_test_acc:.4f}, Test F1: {sage_test_f1:.4f}")

print("\nارزیابی GCN:")
gcn_train_acc, gcn_val_acc, gcn_test_acc, gcn_test_f1 = evaluate_model(gcn_model, data)
print(f"Train Acc: {gcn_train_acc:.4f}, Val Acc: {gcn_val_acc:.4f}, "
      f"Test Acc: {gcn_test_acc:.4f}, Test F1: {gcn_test_f1:.4f}")

# 6. مقایسه مدل‌ها
plt.figure(figsize=(15, 10))

# نمودار خطا
plt.subplot(2, 2, 1)
plt.plot(sage_loss, label='GraphSAGE')
plt.plot(gcn_loss, label='GCN')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# نمودار دقت اعتبارسنجی
plt.subplot(2, 2, 2)
plt.plot(sage_val_acc, label='GraphSAGE')
plt.plot(gcn_val_acc, label='GCN')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# مقایسه F1
plt.subplot(2, 2, 3)
models = ['GraphSAGE', 'GCN']
test_f1 = [sage_test_f1, gcn_test_f1]
plt.bar(models, test_f1, color=['blue', 'orange'])
plt.title('Test F1-Score Comparison')
plt.ylabel('F1-Score')

# مقایسه دقت تست
plt.subplot(2, 2, 4)
test_acc = [sage_test_acc, gcn_test_acc]
plt.bar(models, test_acc, color=['blue', 'orange'])
plt.title('Test Accuracy Comparison')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.savefig('results_comparison.png')
plt.show()

# 7. پیاده‌سازی اضافی: Edge Prediction
class EdgePredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(EdgePredictor, self).__init__()
        self.lin1 = torch.nn.Linear(2 * in_channels, 128)
        self.lin2 = torch.nn.Linear(128, 1)

    def forward(self, z, edge_index):
        src, dst = edge_index
        x = torch.cat([z[src], z[dst]], dim=1)
        x = F.relu(self.lin1(x))
        return torch.sigmoid(self.lin2(x)).squeeze()

def get_embeddings(model, data):
    model.eval()
    with torch.no_grad():
        # استخراج embeddings از لایه اول
        embeddings = model.conv1(data.x, data.edge_index)
        embeddings = F.relu(embeddings)
        return embeddings

print("\nآموزش مدل Edge Prediction با GraphSAGE:")
sage_embeddings = get_embeddings(sage_model, data)
edge_model = EdgePredictor(256).to(device)
optimizer = torch.optim.Adam(edge_model.parameters(), lr=0.01)

# نمونه‌گیری از یالهای منفی
def negative_sampling(edge_index, num_nodes, num_neg_samples=None):
    if num_neg_samples is None:
        num_neg_samples = edge_index.size(1)

    neg_edge_index = torch.randint(0, num_nodes, (2, num_neg_samples), device=device)
    return neg_edge_index

for epoch in range(50):
    edge_model.train()
    optimizer.zero_grad()

    # پیش‌بینی برای یالهای مثبت
    pos_pred = edge_model(sage_embeddings, data.edge_index)
    pos_loss = F.binary_cross_entropy(pos_pred, torch.ones_like(pos_pred))

    # نمونه‌گیری و پیش‌بینی برای یالهای منفی
    neg_edge_index = negative_sampling(data.edge_index, data.num_nodes, num_neg_samples=data.edge_index.size(1))
    neg_pred = edge_model(sage_embeddings, neg_edge_index)
    neg_loss = F.binary_cross_entropy(neg_pred, torch.zeros_like(neg_pred))

    loss = pos_loss + neg_loss
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/50, Loss: {loss.item():.4f}')

# ارزیابی Edge Prediction
edge_model.eval()
with torch.no_grad():
    pos_pred = edge_model(sage_embeddings, data.edge_index)
    neg_edge_index = negative_sampling(data.edge_index, data.num_nodes, num_neg_samples=100000)
    neg_pred = edge_model(sage_embeddings, neg_edge_index)

    # محاسبه دقت
    pos_acc = (pos_pred > 0.5).float().mean()
    neg_acc = (neg_pred < 0.5).float().mean()
    overall_acc = (pos_acc * pos_pred.size(0) + neg_acc * neg_pred.size(0)) / (pos_pred.size(0) + neg_pred.size(0))

    print(f"\nنتایج Edge Prediction:")
    print(f"Positive Accuracy: {pos_acc.item():.4f}")
    print(f"Negative Accuracy: {neg_acc.item():.4f}")
    print(f"Overall Accuracy: {overall_acc.item():.4f}")

Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:03<00:00,  3.61s/it]


Saving...
